<a href="https://colab.research.google.com/github/enigmatized/LookingForAlphaWithPythonNotebooks/blob/main/Crude_time_frame_trading_range_vol_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To get my thoughts on paper.
Right now I just have a bunch of price delta measures.


So price delta in a 'x-time' relation to the next 'x-time'.
There are pockets of more correlated movements.
Specifically around the 8:30s, 9:00, 9:30s

I also wonder what is the relation/correlation of volume with larger price delta.

So I am building a group of minute that are correlated
But what about relative to the volume on that minute


To make a work-able vol bot helper
Command line out to purchase.
Then it basically will buy three contracts.
Trying risk manage that perfectly.

So first is

---



In [30]:
from collections import defaultdict
import requests
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta, time
from urllib.error import HTTPError

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


CSVfiles = []
for file in os.listdir("./"): 
  if file.endswith(".csv"): 
    CSVfiles.append(file)
print(CSVfiles)


unixTimeConverter = lambda x : (datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S')
unixTimeConverterNoDateJustTime = lambda x: (datetime.fromtimestamp(x) - timedelta(hours=4)).time()


allDfs = []


for x in CSVfiles:
  
  dfTemp = pd.read_csv(x)
  dfTemp['estTime']  = dfTemp['time'].apply(unixTimeConverter)
  dfTemp['timeOnly'] = dfTemp['time'].apply(unixTimeConverterNoDateJustTime)
  
  dfTemp['candleHeight']  = dfTemp.apply( lambda row:    max(row['open'],  row['close']) - min(row['open'],  row['close']), axis = 1 )  
  allDfs.append(dfTemp)


#########OH FUCK THIS IS BAD FOR FUTURE
####QUICK HARDCODE FIX for 1minute data set
allDfs[2]  = allDfs[2].tail(-200) 
times_ = []





#Work for 3minute time interval
lsz = []
incur = 0
while incur < 60: 
  lsz.append( (0, incur, 0)  )
  incur +=3
eighthourer = lambda a : (a[0] + 8, a[1], a[2])
ninehourer = lambda  a : (a[0] + 9, a[1], a[2])
times_.append( list(map( eighthourer, lsz))  + list(map( ninehourer, lsz)) )


#List of times tracking
times_.append([ (8,0,0),  (8,30,0), (9,0,0),  (9,30,0), (10,0,0)])


#Work for 1minute time interval
lsz = []
incur = 0
while incur < 60: 
  lsz.append( (0, incur, 0)  )
  incur +=1
eighthourer = lambda a : (a[0] + 8, a[1], a[2])
ninehourer = lambda  a : (a[0] + 9, a[1], a[2])
times_.append( list(map( eighthourer, lsz))  + list(map( ninehourer, lsz)) )


#5min intervals
times_.append([ (8,0,0), 
 (8,5,0),  (8,10,0), (8,15,0),(8,20,0),
 (8,25,0), (8,30,0), (8,35,0), (8,40,0),
 (8,45,0), (8,50,0), (8,55,0), (9,0,0),
 (9,5,0), (9,10,0), (9,15,0),
 (9,20,0), (9,25,0), (9,30,0), (9,35,0),
 (9,40,0), (9,45,0), (9,50,0), (9,55,0),(10,0,0)])
#5minute intervals
times_.append([ (8,0,0), (8,15,0), (8,30,0), (8,45,0), (9,0,0), (9,15,0), (9,30,0), (9,45,0),(10,0,0)])




#make dfs filter out those times

for i, _ in enumerate(allDfs):
  print("-----------------------------------")
  curdeDf = allDfs[i]

  ls_of_dfs = []
  for (a,b,c) in times_[i]: ls_of_dfs.append(curdeDf[curdeDf['timeOnly'].isin([time(a, b, c) ])])

  #Change columnName
  ls_names = []
  for i, (a,b,c) in enumerate(times_[i]):  
    ls_names.append(('candleHeight' + str(a)+str(b)+str(c), str(a)+str(b)+str(c)))
    ls_of_dfs[i]['candleHeight' + str(a)+str(b)+str(c)] =  ls_of_dfs[i]['candleHeight']

  d = { ls_names[i][1] : ls_of_dfs[i][ls_names[i][0]].tolist() for i, v in enumerate(ls_names)}
  fifteen_min_price_range = pd.DataFrame(d)

  #Print out correlation of price width of follow time frames
  for i in range(0, len(ls_names)-1):
    print( str(ls_names[i][1]) + " to " + str(ls_names[i+1][1]) + " Correlation \t",
          fifteen_min_price_range[ls_names[i][1]].corr(fifteen_min_price_range[ls_names[i+1][1]]))



['NYMEX_DL CL1!, 3.csv', 'NYMEX_DL CL1!, 30.csv', 'NYMEX_DL CL1!, 1(8).csv', 'NYMEX_DL CL1!, 5(1).csv', 'NYMEX_DL CL1!, 15.csv']
-----------------------------------
800 to 830 Correlation 	 0.5710480827364774
830 to 860 Correlation 	 0.3231509238393481
860 to 890 Correlation 	 0.6005281858535191
890 to 8120 Correlation 	 0.32536910707641303
8120 to 8150 Correlation 	 0.40420143487776156
8150 to 8180 Correlation 	 0.6984334023974196
8180 to 8210 Correlation 	 0.17661144629295836
8210 to 8240 Correlation 	 0.3213135224586882
8240 to 8270 Correlation 	 -0.006887835818526795
8270 to 8300 Correlation 	 0.3630539092423047
8300 to 8330 Correlation 	 0.839006759682852
8330 to 8360 Correlation 	 0.7803375829956276
8360 to 8390 Correlation 	 0.5778698980486098
8390 to 8420 Correlation 	 0.4697046879262733
8420 to 8450 Correlation 	 0.4832345105825325
8450 to 8480 Correlation 	 0.5760187495832576
8480 to 8510 Correlation 	 0.3944637699454677
8510 to 8540 Correlation 	 0.24683749227450774
8540 to 

<ipython-input-30-5eb7269bf973>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ls_of_dfs[i]['candleHeight' + str(a)+str(b)+str(c)] =  ls_of_dfs[i]['candleHeight']
<ipython-input-30-5eb7269bf973>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ls_of_dfs[i]['candleHeight' + str(a)+str(b)+str(c)] =  ls_of_dfs[i]['candleHeight']
<ipython-input-30-5eb7269bf973>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

800 to 810 Correlation 	 0.3281099697929122
810 to 820 Correlation 	 0.359648371155572
820 to 830 Correlation 	 0.7045872217557843
830 to 840 Correlation 	 0.4954037701106885
840 to 850 Correlation 	 0.24231038515694978
850 to 860 Correlation 	 0.6797032761971848
860 to 870 Correlation 	 0.7661955912454075
870 to 880 Correlation 	 0.5014281450693385
880 to 890 Correlation 	 0.09816410999760512
890 to 8100 Correlation 	 0.19508103287444173
8100 to 8110 Correlation 	 0.7088266727394347
8110 to 8120 Correlation 	 -0.14222358839662258
8120 to 8130 Correlation 	 0.6610931449242999
8130 to 8140 Correlation 	 0.8806143035704864
8140 to 8150 Correlation 	 0.6451920235429236
8150 to 8160 Correlation 	 0.6506122429461035
8160 to 8170 Correlation 	 0.9190643896196988
8170 to 8180 Correlation 	 0.7433472545787202
8180 to 8190 Correlation 	 0.22596092384991334
8190 to 8200 Correlation 	 0.2894888250559068
8200 to 8210 Correlation 	 0.5025306798622177
8210 to 8220 Correlation 	 0.4318709144067792
82

<ipython-input-30-5eb7269bf973>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ls_of_dfs[i]['candleHeight' + str(a)+str(b)+str(c)] =  ls_of_dfs[i]['candleHeight']
<ipython-input-30-5eb7269bf973>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ls_of_dfs[i]['candleHeight' + str(a)+str(b)+str(c)] =  ls_of_dfs[i]['candleHeight']
<ipython-input-30-5eb7269bf973>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [33]:
from collections import defaultdict
import requests
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta, time
from urllib.error import HTTPError

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)


CSVfiles = []
for file in os.listdir("./"): 
  if file.endswith(".csv"): 
    CSVfiles.append(file)
print(CSVfiles)


unixTimeConverter = lambda x : (datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S')
unixTimeConverterNoDateJustTime = lambda x: (datetime.fromtimestamp(x) - timedelta(hours=4)).time()


allDfs = []


for x in CSVfiles:
  
  dfTemp = pd.read_csv(x)
  dfTemp['estTime']  = dfTemp['time'].apply(unixTimeConverter)
  dfTemp['timeOnly'] = dfTemp['time'].apply(unixTimeConverterNoDateJustTime)
  
  dfTemp['candleHeight']  = dfTemp.apply( lambda row:    max(row['open'],  row['close']) - min(row['open'],  row['close']), axis = 1 )  
  allDfs.append(dfTemp)


#########OH FUCK THIS IS BAD FOR FUTURE
####QUICK HARDCODE FIX for 1minute data set
allDfs[2]  = allDfs[2].tail(-200) 
times_ = []





#Work for 3minute time interval
lsz = []
incur = 0
while incur < 60: 
  lsz.append( (0, incur, 0)  )
  incur +=3
eighthourer = lambda a : (a[0] + 8, a[1], a[2])
ninehourer = lambda  a : (a[0] + 9, a[1], a[2])
times_.append( list(map( eighthourer, lsz))  + list(map( ninehourer, lsz)) )


#List of times tracking
times_.append([ (8,0,0),  (8,30,0), (9,0,0),  (9,30,0), (10,0,0)])


#Work for 1minute time interval
lsz = []
incur = 0
while incur < 60: 
  lsz.append( (0, incur, 0)  )
  incur +=1
eighthourer = lambda a : (a[0] + 8, a[1], a[2])
ninehourer = lambda  a : (a[0] + 9, a[1], a[2])
times_.append( list(map( eighthourer, lsz))  + list(map( ninehourer, lsz)) )


#5min intervals
times_.append([ (8,0,0), 
 (8,5,0),  (8,10,0), (8,15,0),(8,20,0),
 (8,25,0), (8,30,0), (8,35,0), (8,40,0),
 (8,45,0), (8,50,0), (8,55,0), (9,0,0),
 (9,5,0), (9,10,0), (9,15,0),
 (9,20,0), (9,25,0), (9,30,0), (9,35,0),
 (9,40,0), (9,45,0), (9,50,0), (9,55,0),(10,0,0)])
#5minute intervals
times_.append([ (8,0,0), (8,15,0), (8,30,0), (8,45,0), (9,0,0), (9,15,0), (9,30,0), (9,45,0),(10,0,0)])




#make dfs filter out those times

for i, _ in enumerate(allDfs):
  print("-----------------------------------")
  curdeDf = allDfs[i]

  #lift df for only that time 
  ls_of_dfs = []
  for (a,b,c) in times_[i]: ls_of_dfs.append(curdeDf[curdeDf['timeOnly'].isin([time(a, b, c) ])])

  #Change columnName
  ls_names = []
  for i, (a,b,c) in enumerate(times_[i]):  
    ls_names.append(('candleHeight' + str(a)+str(b)+str(c), str(a)+str(b)+str(c)))
    ls_of_dfs[i]['candleHeight' + str(a)+str(b)+str(c)] =  ls_of_dfs[i]['candleHeight']

  d = { ls_names[i][1] : ls_of_dfs[i][ls_names[i][0]].tolist() for i, v in enumerate(ls_names)}
  fifteen_min_price_range = pd.DataFrame(d)

  #Print out correlation of price width of follow time frames
  for i in range(0, len(ls_names)-1):

    correlation = fifteen_min_price_range[ls_names[i][1]].corr(fifteen_min_price_range[ls_names[i+1][1]])
    if correlation > 0.57 : print( str(ls_names[i][1]) + " to " + str(ls_names[i+1][1]) + " Correlation \t",correlation)



['NYMEX_DL CL1!, 3.csv', 'NYMEX_DL CL1!, 30.csv', 'NYMEX_DL CL1!, 1(8).csv', 'NYMEX_DL CL1!, 5(1).csv', 'NYMEX_DL CL1!, 15.csv']
-----------------------------------


KeyError: ignored

What can I infer from the below.
Well the reality is I need more data
The 1minute time frame is intresting
It is intresting to see volume z-score relation with price delta for certain time areas.

Still no directional.
But I think higher z-score, and
hgiher-score on the price before may lead to higher vol.


So far I have a strategy forming that is based on
1. High Vol momemtum in specific times (8:00) will likley lead to other high vol movements.
2. There is a correlation with high volumve and high price change 


I want to see the connection between 
(Speed of range intervals) along with trend via Heiskini-candles

ALso correlation of pivot
Like how likely that it will actually change based on small bar
Or something of that aspect.

3. BollingerBand needs to be coded in because there is a slight edge there.

4. 



In [54]:
from collections import defaultdict
import requests
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta, time
from urllib.error import HTTPError

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)


CSVfiles = []
for file in os.listdir("./"): 
  if file.endswith(".csv"): 
    CSVfiles.append(file)
print(CSVfiles)


unixTimeConverter = lambda x : (datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S')
unixTimeConverterNoDateJustTime = lambda x: (datetime.fromtimestamp(x) - timedelta(hours=4)).time()


allDfs = []


for x in CSVfiles:
  
  dfTemp = pd.read_csv(x)
  dfTemp['estTime']  = dfTemp['time'].apply(unixTimeConverter)
  dfTemp['timeOnly'] = dfTemp['time'].apply(unixTimeConverterNoDateJustTime)
  
  dfTemp['candleHeight']  = dfTemp.apply( lambda row:    max(row['open'],  row['close']) - min(row['open'],  row['close']), axis = 1 )  
  allDfs.append(dfTemp)


#########OH FUCK THIS IS BAD FOR FUTURE
####QUICK HARDCODE FIX for 1minute data set
allDfs[2]  = allDfs[2].tail(-200) 
times_ = []





#Work for 3minute time interval
lsz = []
incur = 0
while incur < 60: 
  lsz.append( (0, incur, 0)  )
  incur +=3
eighthourer = lambda a : (a[0] + 8, a[1], a[2])
ninehourer = lambda  a : (a[0] + 9, a[1], a[2])
times_.append( list(map( eighthourer, lsz))  + list(map( ninehourer, lsz)) )


#List of times tracking
times_.append([ (8,0,0),  (8,30,0), (9,0,0),  (9,30,0), (10,0,0)])


#Work for 1minute time interval
lsz = []
incur = 0
while incur < 60: 
  lsz.append( (0, incur, 0)  )
  incur +=1
eighthourer = lambda a : (a[0] + 8, a[1], a[2])
ninehourer = lambda  a : (a[0] + 9, a[1], a[2])
times_.append( list(map( eighthourer, lsz))  + list(map( ninehourer, lsz)) )


#5min intervals
times_.append([ (8,0,0), 
 (8,5,0),  (8,10,0), (8,15,0),(8,20,0),
 (8,25,0), (8,30,0), (8,35,0), (8,40,0),
 (8,45,0), (8,50,0), (8,55,0), (9,0,0),
 (9,5,0), (9,10,0), (9,15,0),
 (9,20,0), (9,25,0), (9,30,0), (9,35,0),
 (9,40,0), (9,45,0), (9,50,0), (9,55,0),(10,0,0)])
#5minute intervals
times_.append([ (8,0,0), (8,15,0), (8,30,0), (8,45,0), (9,0,0), (9,15,0), (9,30,0), (9,45,0),(10,0,0)])


import numpy as np
from scipy.stats import norm 


# norm(0, 1).cdf(-np.absolute(zscore)) * 2 



#make dfs filter out those times

for i, v in enumerate(allDfs):

  print("-----------------------------------")
  curdeDf = allDfs[i]

  
  ls_of_dfs = []
  for (a,b,c) in times_[i]: 
    #Create a filiteed df with the time frame I am intrested in studying
    nu_df = curdeDf[curdeDf['timeOnly'].isin([time(a, b, c) ])]
    
    #This gets the average volume and standard deviation for this time frame
    volume_average = nu_df["Volume"].describe()["mean"]
    volume_std     = nu_df["Volume"].describe()["std"]

    warnings.filterwarnings('ignore')
    nu_df["volume_z"]  = nu_df["Volume"].apply(lambda row:  ( row - volume_average  ) / volume_std)
    warnings.filterwarnings('ignore')
    nu_df["volume_p"]  = nu_df["volume_z"].apply(lambda zscore:  norm(volume_average, volume_std).cdf(-np.absolute(zscore)) * 2  )


    candleHeight_average = nu_df["candleHeight"].describe()["mean"]
    candleHeight_std = nu_df["candleHeight"].describe()["std"]

    nu_df["candleHeight_z"]  = nu_df["candleHeight"].apply(lambda row:  ( row - candleHeight_average  ) / candleHeight_std)
    
    nu_df["candleHeight_p"]  = nu_df["candleHeight_z"].apply(lambda zscore:  norm(candleHeight_average, candleHeight_std).cdf(-np.absolute(zscore)) * 2  )

    print(time(a, b, c),  "correlation of probability", nu_df["candleHeight_z"].corr(nu_df["volume_z"])  )
    
    ls_of_dfs.append(nu_df)





['NYMEX_DL CL1!, 3.csv', 'NYMEX_DL CL1!, 30.csv', 'NYMEX_DL CL1!, 1(8).csv', 'NYMEX_DL CL1!, 5(1).csv', 'NYMEX_DL CL1!, 15.csv']
-----------------------------------
08:00:00 correlation of probability 0.5479200080423048
08:03:00 correlation of probability 0.5699542256145483
08:06:00 correlation of probability 0.6347939531013879
08:09:00 correlation of probability 0.6517145002034425
08:12:00 correlation of probability 0.6080392184437434
08:15:00 correlation of probability 0.665825515016706
08:18:00 correlation of probability 0.6549918243376337
08:21:00 correlation of probability 0.5129420658230183
08:24:00 correlation of probability 0.5337000233476101
08:27:00 correlation of probability 0.27312308433815263
08:30:00 correlation of probability 0.8897126920589682
08:33:00 correlation of probability 0.6958023172405591
08:36:00 correlation of probability 0.8266896230878362
08:39:00 correlation of probability 0.571274906527724
08:42:00 correlation of probability 0.5222424813383567
08:45:00 co

In [ ]:
#Recrusively?
#Or just look at the vol patterns in different time frames.
#Vol correlations
#so 8 to 8:30 correlation
#then 8, 8:15, 8:30, 8:45
#Then Vol avereages
#For each min
#and each 5min 
#

In [ ]:
#What is the point of this work?
#I beleieve it is to see if 8-8:30 was high vol
#I have 70% chance today will be higher vol.
#Still have to pick directionally
#This work is just to tell me there is movement in the market

#Bouncing towards
#And Bouncing away from levels


#What is the chance if it goes x distance that it goes 2x distance?
#Or what is the changce of x green, then next being green?
#




In [40]:
allDfs[0].head()["Volume"].describe()

count      5.00000
mean     336.20000
std      299.01622
min      167.00000
25%      172.00000
50%      176.00000
75%      305.00000
max      861.00000
Name: Volume, dtype: float64